# Análise Exploratória de Dados (EDA) - Previsão de Volatilidade

## Objetivo

O objetivo deste notebook é conduzir uma análise exploratória profunda no dataset preparado para a modelagem de previsão de volatilidade do par BTC/USDT. A EDA é dividida em três partes:

1.  **Análise da Variável-Alvo (`target_volatility`):** Compreender sua distribuição, comportamento temporal e autocorrelação.
2.  **Análise Univariada das Features:** Investigar as características individuais das variáveis preditoras.
3.  **Análise Bivariada e Multivariada:** Descobrir as relações entre as features e a variável-alvo e entre as próprias features.

Os insights obtidos aqui guiarão a estratégia de modelagem, seleção de features e validação do modelo.

In [ ]:
# ==============================================================================
# 1. SETUP INICIAL E CARREGAMENTO DOS DADOS
# ==============================================================================

# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Importando nossos módulos customizados (assegure que estão no PATH ou no mesmo diretório)
from data_downloader import fetch_ohlcv_data
from feature_engineering import create_target_variable, create_all_features

# Configurações de estilo para os gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Parâmetros para carregamento (ajuste conforme necessário)
SYMBOL = 'BTC/USDT'
START_DATE = '2022-01-01'  # 3 anos de dados para uma análise robusta
TIMEFRAME = '1h'

# Pipeline de criação do dataset
df_raw = fetch_ohlcv_data(SYMBOL, START_DATE, TIMEFRAME)
df_target = create_target_variable(df_raw)
df_features = create_all_features(df_target)

# Dataset final limpo, pronto para a EDA e modelagem
df_eda = df_features.dropna().copy()

print(f"Dataset pronto para EDA. Dimensões: {df_eda.shape}")
df_eda.head()


---
## Parte 1: Análise da Variável-Alvo (`target_volatility`)

Nesta seção, focamos exclusivamente na variável que queremos prever. Compreender suas propriedades é o passo mais crítico antes da modelagem.

In [ ]:
# 1.1 Análise de Distribuição
plt.figure(figsize=(14, 6))
sns.histplot(df_eda['target_volatility'], kde=True, bins=50)
plt.title('Distribuição da Volatilidade Alvo (Anualizada)', fontsize=16)
plt.xlabel('Volatilidade Anualizada')
plt.ylabel('Frequência')

skewness = df_eda['target_volatility'].skew()
kurtosis = df_eda['target_volatility'].kurt()

plt.axvline(df_eda['target_volatility'].mean(), color='r', linestyle='--', label=f'Média')
plt.axvline(df_eda['target_volatility'].median(), color='g', linestyle='-', label=f'Mediana')
plt.legend()
plt.show()

print(f"Assimetria (Skewness): {skewness:.4f}")
print(f"Curtose (Kurtosis): {kurtosis:.4f}")


---
## Parte 2: Análise Univariada das Features Preditivas

Agora, vamos analisar as características de algumas das nossas features mais importantes.

In [ ]:
# 2.1 Estatísticas Descritivas
pd.set_option('display.max_columns', None)
display(df_eda.describe())


In [ ]:
# 2.2 Distribuição das Features Chave
key_features = ['vol_realizada_24h', 'vol_ratio_24_168', 'retorno_acumulado_24h', 'volume_ratio_1_24', 'atr_24h']

plt.figure(figsize=(15, 8))
# Para evitar problemas caso alguma feature não exista, filtrar as presentes
key_present = [c for c in key_features if c in df_eda.columns]
if len(key_present) == 0:
    print("Nenhuma das key_features está presente no df_eda. Verifique nomes de colunas.")
else:
    sns.boxplot(data=df_eda[key_present])
    plt.title('Distribuição de Features Chave (Box Plot)', fontsize=16)
    plt.xticks(rotation=45)
    plt.show()


---
## Parte 3: Análise Bivariada e Multivariada (Relações)

Nesta seção, investigamos como as features se relacionam com o alvo e entre si.

In [ ]:
# 3.1 Correlação entre Features e o Alvo
# Calcula a matriz de correlação
corr_matrix = df_eda.corr()

# Foca nas correlações com a variável-alvo
if 'target_volatility' in corr_matrix.columns:
    corr_target = corr_matrix['target_volatility'].sort_values(ascending=False)
    plt.figure(figsize=(8, 10))
    sns.heatmap(corr_target.to_frame(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlação das Features com a Volatilidade Alvo')
    plt.show()
else:
    print("Coluna 'target_volatility' não encontrada na matriz de correlação.")


In [ ]:
# 3.2 Análise de Multicolinearidade
plt.figure(figsize=(18, 15))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)
plt.title('Matriz de Correlação Completa (Multicolinearidade)', fontsize=16)
plt.show()


In [ ]:
# 3.3 Scatter Plots: Top Features vs. Target
if 'target_volatility' in corr_matrix.columns:
    top_features = corr_target.index[1:5]  # Pega as 4 features mais correlacionadas (excluindo o próprio alvo)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Relação das Top Features com a Volatilidade Alvo', fontsize=18)
    for i, feature in enumerate(top_features):
        ax = axes[i//2, i%2]
        if feature in df_eda.columns:
            sns.scatterplot(x=df_eda[feature], y=df_eda['target_volatility'], ax=ax, alpha=0.3, s=10)
            ax.set_title(f'Target vs. {feature}')
        else:
            ax.text(0.5, 0.5, f'Feature {feature} não encontrada', ha='center')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
else:
    print("Não foi possível calcular top_features porque 'target_volatility' não está presente.")


In [ ]:
# 3.4 Análise de Features Temporais
# Garantir colunas horárias existam
if 'hora_do_dia' in df_eda.columns and 'dia_da_semana' in df_eda.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    sns.boxplot(x='hora_do_dia', y='target_volatility', data=df_eda, ax=ax1)
    ax1.set_title('Volatilidade Alvo por Hora do Dia')
    sns.boxplot(x='dia_da_semana', y='target_volatility', data=df_eda, ax=ax2)
    # Ajuste rótulos dia da semana caso sejam numéricos (0-6)
    try:
        ax2.set_xticklabels(['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom'])
    except Exception:
        pass
    ax2.set_title('Volatilidade Alvo por Dia da Semana')
    plt.tight_layout()
    plt.show()
else:
    print("As colunas 'hora_do_dia' e/ou 'dia_da_semana' não foram encontradas em df_eda.")


---
## Conclusão da EDA e Próximos Passos

1. **Validação da Premissa:** A análise costuma confirmar a forte persistência da volatilidade, validando a hipótese central do projeto.
2. **Seleção de Features:** Manteremos o conjunto completo para a primeira versão do modelo e usaremos feature importance para refinar.
3. **Estratégia de Validação:** Utilizar validação cruzada baseada em tempo (Time Series CV) ou divisão temporal treino-teste para evitar vazamento.
4. **Baseline do Modelo:** Um modelo ingênuo que prevê `target_volatility(t) = vol_realizada_24h(t)` é um bom baseline. Próximo passo: preparar dados e treinar LightGBM.